In [1]:
import pandas as pd
import numpy as np
import nltk
import pickle
from contextlib import contextmanager
import copy
import os
import re
import string
import time
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
@contextmanager
def timer(msg):
    t0 = time.time()
    print(f'[{msg}] start.')
    yield
    elapsed_time = time.time() - t0
    print(f'[{msg}] done in {elapsed_time / 60:.2f} min.')

In [3]:
misspell_dict = {"aren't": "are not", "can't": "cannot", "couldn't": "could not",
                 "didn't": "did not", "doesn't": "does not", "don't": "do not",
                 "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                 "he'd": "he would", "he'll": "he will", "he's": "he is",
                 "i'd": "I had", "i'll": "I will", "i'm": "I am", "isn't": "is not",
                 "it's": "it is", "it'll": "it will", "i've": "I have", "let's": "let us",
                 "mightn't": "might not", "mustn't": "must not", "shan't": "shall not",
                 "she'd": "she would", "she'll": "she will", "she's": "she is",
                 "shouldn't": "should not", "that's": "that is", "there's": "there is",
                 "they'd": "they would", "they'll": "they will", "they're": "they are",
                 "they've": "they have", "we'd": "we would", "we're": "we are",
                 "weren't": "were not", "we've": "we have", "what'll": "what will",
                 "what're": "what are", "what's": "what is", "what've": "what have",
                 "where's": "where is", "who'd": "who would", "who'll": "who will",
                 "who're": "who are", "who's": "who is", "who've": "who have",
                 "won't": "will not", "wouldn't": "would not", "you'd": "you would",
                 "you'll": "you will", "you're": "you are", "you've": "you have",
                 "'re": " are", "wasn't": "was not", "we'll": " will", "tryin'": "trying"}

def _get_misspell(misspell_dict):
    misspell_re = re.compile('(%s)' % '|'.join(misspell_dict.keys()))
    return misspell_dict, misspell_re

def replace_typical_misspell(text):
    misspellings, misspellings_re = _get_misspell(misspell_dict)
    def replace(match):
        return misspellings[match.group(0)]
    return misspellings_re.sub(replace, text)
    
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']',
          '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^',
          '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█',
          '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶',
          '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼',
          '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
          'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪',
          '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

def clean_text(x):
    x = str(x)
    for punct in puncts + list(string.punctuation):
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    return re.sub(r'\d+', ' ', x)

In [19]:
train_data = '/Users/kelvin/Toxicity/data/train.csv'
data = pd.read_csv(train_data)

In [20]:
def load_and_prec():
    train = pd.read_csv(train_data)
    #train = train.dropna(axis=0)
    # lower
    train['comment_text'] = train['comment_text'].str.lower()
    # clean misspellings
    train['comment_text'] = train['comment_text'].apply(replace_typical_misspell)
    # clean the text
    train['comment_text'] = train['comment_text'].apply(clean_text)
    # clean numbers
    train['comment_text'] = train['comment_text'].apply(clean_numbers)
    # strip
    train['comment_text'] = train['comment_text'].str.strip()
    # replace blank with nan
    train['comment_text'].replace('', np.nan, inplace=True)
    # fill up the missing values
    train['comment_text'] = pd.Series(train['comment_text'].fillna('_##_').values)
    return train

In [21]:
with timer('load data'):
    train = load_and_prec()

[load data] start.
[load data] done in 2.32 min.


In [22]:
with timer('load data'):
    # Text preprocessing steps - remove numbers, captial letters and punctuation
    alphabetic = lambda x: re.sub('\w*\d\w*', ' ', x)
    punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
    # text
    text = train['comment_text'].map(alphabetic).map(punc_lower)
    text.head() 

[load data] start.
[load data] done in 1.71 min.


In [23]:
train["comment_text"] = text
train.rename({'homosexual_gay_or_lesbian':'gay_lesbian', 'intellectual_or_learning_disability':'learning_disability', 'psychiatric_or_mental_illness':'mental_illness', 'other_race_or_ethnicity':'other_race'}, axis=1, inplace=True)

In [24]:
with timer('labels'):
    train['y'] = np.where(train['target'] >= 0.5, 1, 0)
    train.y.value_counts(normalize=True)*100

[labels] start.
[labels] done in 0.00 min.


In [25]:
train.shape

(1804874, 46)

In [26]:
def preprocess(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text
    data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
    return data

In [37]:
text_cleaned = preprocess(train.comment_text)

In [38]:
train['comment_text'] = text_cleaned

In [41]:
data_clean_short = train.dropna()

In [46]:
data_clean_short.to_csv('cleaned_short.csv', index=False)

In [47]:
data_clean = pd.read_csv('cleaned_short.csv')

In [45]:
train.to_csv('cleaned_long.csv', index=False)